In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/code/phasor_julia`


In [2]:
include("module.jl")
using .PhasorNetworks

┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243


┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243


In [3]:
using LinearAlgebra: triu, diagm

In [4]:
using Statistics: std

In [5]:
chance_level(1024, 200)

0.02160992606191345

In [25]:
function generate_er_graph(n::Int, p::Real, self_loops::Bool = false)
    adj = rand(Float64, (n, n)) .< p
    #make undirected
    adj = triu(adj)
    adj =  (adj .+ adj') .> 0

    if !self_loops
        #remove self-loops
        for i in 1:n
            adj[i,i] = 0
        end
    end

    return adj
end

generate_er_graph (generic function with 2 methods)

In [26]:
g1 = generate_er_graph(20, 0.1)

20×20 BitMatrix:
 0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0

In [27]:
edges = findall(g1)

46-element Vector{CartesianIndex{2}}:
 CartesianIndex(4, 1)
 CartesianIndex(6, 1)
 CartesianIndex(19, 1)
 CartesianIndex(20, 2)
 CartesianIndex(8, 3)
 CartesianIndex(9, 3)
 CartesianIndex(10, 3)
 CartesianIndex(11, 3)
 CartesianIndex(1, 4)
 CartesianIndex(8, 4)
 ⋮
 CartesianIndex(4, 19)
 CartesianIndex(10, 19)
 CartesianIndex(13, 19)
 CartesianIndex(17, 19)
 CartesianIndex(2, 20)
 CartesianIndex(6, 20)
 CartesianIndex(9, 20)
 CartesianIndex(11, 20)
 CartesianIndex(18, 20)

In [28]:
e1 = edges[1]

CartesianIndex(4, 1)

In [29]:
function graph_to_vector(graph::AbstractMatrix, nd::Int)
    @assert size(graph,1) == size(graph,2) "Takes an adjacency matrix as the input"
    n = size(graph,1)
    #create vectors representing the nodes
    node_values = random_symbols((n, nd))
    #get cartesian coordinates representing each edge
    edges = findall(graph)
    n_edges = length(edges)
    edge_values = zeros(Float64, n_edges, nd)

    #iterate through the edges
    for (i, edge) in enumerate(edges)
        tx = edge[1]
        rx = edge[2]

        tx_symbol = node_values[tx,:]
        rx_symbol = node_values[rx,:]
        #create a representation for that edge by binding its incident nodes
        edge_symbol = PhasorNetworks.bind(tx_symbol, rx_symbol)
        edge_values[i,:] = edge_symbol
    end

    #combine the edges in the graph to the single embedding via bundling
    graph_embedding = bundle(edge_values, dims=1)
    return node_values, graph_embedding
end

graph_to_vector (generic function with 1 method)

In [30]:
nv1, gv1 = graph_to_vector(g1, 1024)

([0.6192258596420288 0.31488358974456787 … -0.053089261054992676 0.5775110721588135; -0.09060060977935791 0.6215229034423828 … -0.44997072219848633 -0.32227563858032227; … ; 0.21948456764221191 0.6484586000442505 … 0.8004494905471802 -0.9631133079528809; 0.4952505826950073 -0.9375089406967163 … 0.051482439041137695 -0.7570844888687134], [0.1999413365065328 -0.44744670235909284 … -0.38399521908670325 -0.36504054346663756])

In [31]:
nv1 |> size

(20, 1024)

In [32]:
gv1

1×1024 Matrix{Float64}:
 0.199941  -0.447447  -0.342239  0.43161  …  -0.269928  -0.383995  -0.365041

In [33]:
zeros(Int64, 3,3)

3×3 Matrix{Int64}:
 0  0  0
 0  0  0
 0  0  0

In [34]:
e = nv1[1:1,:]

1×1024 Matrix{Float64}:
 0.619226  0.314884  0.45361  0.424709  …  -0.865675  -0.0530893  0.577511

In [35]:
e2 = eachslice(nv1, dims=1)[1]

1024-element view(::Matrix{Float64}, 1, :) with eltype Float64:
  0.6192258596420288
  0.31488358974456787
  0.45361006259918213
  0.4247094392776489
  0.048892974853515625
 -0.897257924079895
  0.8199940919876099
  0.7890224456787109
  0.018862128257751465
 -0.845494270324707
  ⋮
 -0.38997113704681396
  0.7367422580718994
  0.6467392444610596
 -0.8650505542755127
  0.2849041223526001
  0.5926374197006226
 -0.865674614906311
 -0.053089261054992676
  0.5775110721588135

In [36]:
q = unbind(gv1, e)

1×1024 Matrix{Float64}:
 -0.419285  -0.76233  -0.795849  0.0069005  …  0.595746  -0.330906  -0.942552

In [37]:
q = unbind(gv1, reshape(e2, (1,:)))

1×1024 Matrix{Float64}:
 -0.419285  -0.76233  -0.795849  0.0069005  …  0.595746  -0.330906  -0.942552

In [38]:
similarity_outer(q, nv1, dims = 1) |> vec

20-element Vector{Float64}:
 -5.739850983653495e-5
  0.014994900397499334
 -0.026325921486153907
  0.1341276604866228
  0.009383427563616209
  0.15146370048990204
  0.04108899631156734
 -0.029100052216749526
 -0.007813304275803281
  0.0038315523632059564
 -0.00980438638904436
 -0.068700703798954
  0.001678597506371525
 -0.00860024256799912
 -0.022026601464654356
  0.010261442142451865
  0.017846635810013584
  0.005093642236488586
  0.15965314472533376
 -0.0023130963542708433

In [39]:
function query_edges(graph::AbstractMatrix, nodes::AbstractMatrix, threshold::Real = 3.0)
    n = size(nodes, 1)
    nd = size(graph, 2)

    sim_threshold = threshold * chance_level(nd, 200)

    adj_rec = zeros(Int64, n, n)
    for (i,node) in enumerate(eachslice(nodes, dims=1))
        #add a dimension for consistency
        node = reshape(node, (1, :))
        query = unbind(graph, node)
        s = similarity_outer(query, nodes, dims=1) |> vec
        adj_node = s .> sim_threshold
        adj_rec[i,:] = adj_node
    end

    return adj_rec
end

query_edges (generic function with 2 methods)

In [40]:
rec1 = query_edges(gv1, nv1, 3)

20×20 Matrix{Int64}:
 0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  

In [41]:
rec1 .- g1

20×20 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  